In [56]:
import os
import subprocess
from glob import glob
import math
from tqdm import tqdm

##### Helper functions

In [66]:
def excute_cmd(command):
    # excute the command
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True)

    # Check the return code to see if the command was successful
    if result.returncode == 0:
        # print("Command executed successfully.")
        # print("Output:")
        return result.stdout
    else:
        print(f"Command failed with an error: {command}")
        print(result.stderr)
        return result.stderr

def create_directory_if_not_exists(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [58]:
print(excute_cmd('elastix --help'))

elastix version: 5.000

elastix registers a moving image to a fixed image.
The registration-process is specified in the parameter file.
  --help, -h displays this message and exit
  --version  output version information and exit
  --extended-version  output extended version information and exit

Call elastix from the command line with mandatory arguments:
  -f        fixed image
  -m        moving image
  -out      output directory
  -p        parameter file, elastix handles 1 or more "-p"

Optional extra commands:
  -fMask    mask for fixed image
  -mMask    mask for moving image
  -t0       parameter file for initial transform
  -priority set the process priority to high, abovenormal, normal (default),
            belownormal, or idle (Windows only option)
  -threads  set the maximum number of threads of elastix

The parameter-file must contain all the information necessary for elastix to run properly. That includes which metric to use, which optimizer, which transform, etc. It must 

#### For us to find the best reference frame (fixed image) among the given datasets, we can measure some image quality and similarity metrics such as mutual information, signal to noise ratio (SNR), image sharpness, etc..

#### To start with the similarity metrics, as the mutual information, we need to register all images with all possible combinations to see how well images align with all combinations after the registration. The registration will be done using elastix command line, and some helper functions will be developed to ease the process of excuting and registering all images. 

For elastix, we need to specify the parameters files, which for our case will be both Par0010affine.txt and Par0010bspline.txt

In [59]:
reg_params = '-p "../Par0010affine.txt" -p "../Par0010bspline.txt"'

Lets get all images that we have for training

In [60]:
training_path = '../training-set/training-images'

training_volumes = sorted(glob(os.path.join(training_path, "*.nii.gz"), recursive=True))
training_volumes

# substract the length as we won't register the images with themselves
print('Total registrations: ', len(training_volumes)*len(training_volumes)-len(training_volumes)) 

Total registrations:  210


In [62]:
for fixed in tqdm(training_volumes):
    for moving in training_volumes:
        if fixed == moving: # to not register the image with its self
            continue
            
        # We register here, every image will be registered with all combinations except its self
        reg_fixed  = fixed
        reg_moving = moving

        # Get the names of the fixed and moving images for the output directory, names without the file extensions
        reg_fixed_name  = reg_fixed.split("\\")[-1].split(".")[0]
        reg_moving_name = reg_moving.split("\\")[-1].split(".")[0]
        
        # create an output folder using the image name in the output/images directory
        # output folders structures
        # output_{fixed_img_name}/{moving_image_name}/
        output_dir = f'output_{reg_fixed_name}/{reg_moving_name}'

        # creates the output directory
        create_directory_if_not_exists(output_dir)

        # create elastix command line
        command_line = f'elastix -f "{reg_fixed}" -m "{reg_moving}" {reg_params} -out "{output_dir}"'

        # run elastix on all combinations
        excute_cmd(command_line)

 13%|█▋           | 2/15 [14:57<1:37:02, 447.92s/it]

Command failed with an error: elastix -f "../training-set/training-images\1002.nii.gz" -m "../training-set/training-images\1006.nii.gz" -p "../Par0010affine.txt" -p "../Par0010bspline.txt" -out "output_1002/1006"
Command failed with an error: elastix -f "../training-set/training-images\1002.nii.gz" -m "../training-set/training-images\1017.nii.gz" -p "../Par0010affine.txt" -p "../Par0010bspline.txt" -out "output_1002/1017"


 20%|██▌          | 3/15 [22:05<1:27:47, 438.97s/it]

Command failed with an error: elastix -f "../training-set/training-images\1006.nii.gz" -m "../training-set/training-images\1002.nii.gz" -p "../Par0010affine.txt" -p "../Par0010bspline.txt" -out "output_1006/1002"


 27%|███▍         | 4/15 [29:12<1:19:35, 434.09s/it]

...

 80%|█████████▌  | 12/15 [1:29:20<23:04, 461.54s/it]

Command failed with an error: elastix -f "../training-set/training-images\1015.nii.gz" -m "../training-set/training-images\1002.nii.gz" -p "../Par0010affine.txt" -p "../Par0010bspline.txt" -out "output_1015/1002"


 93%|███████████▏| 14/15 [1:44:46<07:42, 462.62s/it]

Command failed with an error: elastix -f "../training-set/training-images\1036.nii.gz" -m "../training-set/training-images\1002.nii.gz" -p "../Par0010affine.txt" -p "../Par0010bspline.txt" -out "output_1036/1002"


100%|████████████| 15/15 [1:53:13<00:00, 452.87s/it]


Some commands has failed, repeating them manually to make sure all combinations were registered

In [67]:
# create elastix command line
command_line = r'elastix -f "../training-set/training-images/1002.nii.gz" -m "../training-set/training-images/1006.nii.gz" -p "../Par0010affine.txt" -p "../Par0010bspline.txt" -out "output_1002/1006"'

# run elastix on all combinations
print(excute_cmd(command_line))

Command failed with an error: elastix -f "../training-set/training-images/1002.nii.gz" -m "../training-set/training-images/1006.nii.gz" -p "../Par0010affine.txt" -p "../Par0010bspline.txt" -out "output_1002/1006"


